In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

biloxx_nutritiondataset_path = kagglehub.dataset_download('biloxx/nutritiondataset')

print('Data source import complete.')


In [ ]:
# =============================
# FULL RAG NOTEBOOK (single cell)
# =============================
# 1) Install dependencies
!pip install -q \
    langchain \
    langchain-community \
    faiss-cpu \
    sentence-transformers \
    transformers \
    pymupdf \
    accelerate \
    bitsandbytes \
    torch \
    tqdm

# 2) Imports
import os
import fitz   # PyMuPDF
from tqdm.auto import tqdm

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# 3) Auto-detect PDFs in ../input
pdf_files = []
for root, dirs, files in os.walk("../input"):
    for f in files:
        if f.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(root, f))

if not pdf_files:
    raise FileNotFoundError("No PDF files found in ../input. Please attach a dataset with PDFs to the notebook.")

print("Found PDF files:")
for p in pdf_files:
    print(" -", p)

# 4) Extract text from all PDFs (concatenate)
all_text = ""
for pdf_path in pdf_files:
    print(f"\nExtracting from: {pdf_path}")
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"  Failed to open {pdf_path}: {e}")
        continue
    for page in doc:
        # get_text() is simple and usually good; if you need layout-aware extraction, adapt here
        all_text += page.get_text()
    doc.close()

print(f"\nTotal extracted characters: {len(all_text)}")
if len(all_text) < 50:
    raise ValueError("Extracted very little text — check that the PDFs actually contain extractable text (not scanned images).")

# 5) Split text into chunks for retrieval
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
texts = splitter.split_text(all_text)
print("Total chunks:", len(texts))

# Optional: show a sample chunk
print("\n--- Sample chunk ---\n")
print(texts[0][:500])
print("\n--- End sample ---\n")

# 6) Create embeddings and FAISS vector store
print("Creating embeddings... (this may take a few minutes)")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# Build FAISS DB
db = FAISS.from_texts(texts, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})
print("Vector store ready (FAISS).")

# 7) Load local LLM (google/flan-t5-base)
#    NOTE: This model is CPU-friendly but still may take time to download.
model_name = "google/flan-t5-base"
print(f"Loading model {model_name} ... (may take a minute)")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=pipe)
print("Model loaded.")

# 8) Build RetrievalQA RAG chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",   # 'stuff' works for shorter contexts; change if you want other chain types
)
print("RAG pipeline ready. You can now ask questions.")

# 9) Interactive query loop (type 'exit' to stop)
print("\nType queries to ask about the PDF(s). Type 'exit' to quit.")
while True:
    try:
        query = input("\nYour question: ").strip()
    except Exception:
        # In some Kaggle environments input() may not behave interactively - handle gracefully
        print("Interactive input not available in this environment. To run a one-off query, set 'query' variable and call qa.run(query).")
        break
    if query.lower() in ("exit", "quit"):
        print("Exiting interactive loop.")
        break
    if not query:
        print("Please type a question or 'exit'.")
        continue

    try:
        # Run retrieval + generation
        answer = qa.run(query)
        print("\nAnswer:\n", answer)
    except Exception as e:
        print("Error during QA:", e)
        # fall back to showing top retrieved chunks (useful for debugging)
        try:
            docs = retriever.get_relevant_documents(query)
            print("\nTop retrieved chunks (for debugging):")
            for i, d in enumerate(docs[:3], 1):
                print(f"\n--- chunk {i} ---\n{d.page_content[:800]}\n")
        except Exception as e2:
            print("Also failed to retrieve debug docs:", e2)

# 10) Optional: save FAISS index to disk for reuse
save_dir = "/kaggle/working/faiss_index"
os.makedirs(save_dir, exist_ok=True)
db.save_local(save_dir)
print(f"FAISS index saved to: {save_dir}")
print("Done.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 50.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 27.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━

2025-10-19 20:01:59.712401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760904119.990138      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760904120.068247      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found PDF files:
 - ../input/nutritiondataset/nutrition.pdf

Extracting from: ../input/nutritiondataset/nutrition.pdf

Total extracted characters: 3076893
Total chunks: 3582

--- Sample chunk ---

VITAMIN
VITAMIN
PANTOTHENIC
LIFE STAGE
VITAMIN A
VITAMIN C
VITAMIN D
VITAMIN E
VITAMIN K
THIAMIN
RIBOFLAVIN
NIACIN
B6
FOLATE
B12
ACID
BIOTIN
CHOLINEg
GROUP
(G/DAY)a
(MG/DAY)
(G/DAY)b,c
(MG/DAY)d
(G/DAY)
(MG/DAY)
(MG/DAY)
(MG/DAY)e
(MG/DAY)
(G/DAY)f
(G/DAY)
(MG/DAY)
(G/DAY)
(MG/DAY)
INFANTS
0–6 mo
400*
40*
5*
4*
2.0*
0.2*
0.3*
2*
0.1*
65*
0.4*
1.7*
5*
125*
7–12 mo
500*
50*
5*
5*
2.5*
0.3*
0.4*
4*
0.3*
80*
0.5*
1.8*
6*
150*
CHILDREN
1–3 y
300
15
5*
6
30*
0.5
0.5
6
0.5
150
0.9
2*
8*
200*
4–8 

--- End sample ---

Creating embeddings... (this may take a few minutes)


/tmp/ipykernel_37/3478752661.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

/tmp/ipykernel_37/3478752661.py:105: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(query)
Token indices sequence length is longer than the specified maximum sequence length for this model (1301 > 512). Running this sequence through the model will result in indexing errors



Answer:
 100g of cooked quinoa.



Your question:  What vitamins are especially important for bone health?



Answer:
 Vitamin D


KeyboardInterrupt: Interrupted by user